[![Cython](cython.png)](http://cython.org)



* Built to be a Python+C language for high performance computations
* Performance computation space in competition with Numba
* Due to design, also makes binding easy
* Easy to customize result
* Can write Python 2 or 3, regardless of calling language
* Works with both C and C++ (as an option)
* Good development (Version 3.0 in advanced alpha stage!)
* Used in SciPy and many other places.

Downsides:

* Requires learning a new(ish) language
* Have to think with three hats
* *Very* verbose
* Not really the best binding tool. Not really the best acceleration tool.

# Using Cython in a notebook

Cython comes with an IPython extension, making it simple to use in a notebook. Let's give it a try:

In [ ]:
%load_ext Cython

In [ ]:
%%cython -a

cimport cython

def square(x: cython.int):
    v = x*x
    return v


In [ ]:
square(4)

Try adding `-a` to the magic; you can see the generated C code.

The rules of the Cython syntax are simple:

* Follow Python syntax where something is valid in Python: `v = x*x` deduces the type of v
* Otherwise, use C syntax: `int x` gives x a type
* Unless the syntax addition to Python happened after Cython: `x: int` is valid Python but is not inferred by Cython. It does understand `cython.int` though.
* And unless the syntax is invalid in Python: You can't dereference a pointer with `*x`; but only with `x[0]`.
* And if concepts don't match, invent something completely bizarre. Type Fusion is like C++ templates, sort of.

Okay, not that simple. Yes, it's a new language to learn.

# Aside: Speed comparison Python, Cython, [Numba](https://numba.pydata.org)

In [ ]:
def fp(x):
    for _ in range(10_000_000):
        x=x+1
    return x

In [ ]:
%%time
fp(1)

In [ ]:
%%cython -a
def fc(int x):
    for _ in range(10_000_000):
        x=x+1
    return x

In [ ]:
%%timeit
fc(23)

In [ ]:
import numba
@numba.jit(numba.int64(numba.int64))
def fn(x):
    for _ in range(10_000_000):
        x=x+1
    return x

In [ ]:
%time
fn(41)

In [ ]:
%%timeit
fn(41)

> ## Warning: this is just about the only speed test I can think of that shows Cython faster than Numba.
> 
> The reason is that Cython pretty much makes a pure CPython object, while there is a tiny bit of bookkeeping that Numba does.

By now, have you detected anything odd about our "speed" test? Are you happy with it?

Let's check the output:

In [ ]:
print(fn.inspect_asm((numba.int64,)))

I'm not great at assembly, but I don't see a loop! Our compiler was too smart!

# Binding with [Cython](https://cython.org): C++

In [ ]:
%%writefile SimpleClass.hpp
#pragma once

class Simple {
    int x;
    
public:

    Simple(int x): x(x) {}
    
    int get() const {
        return x;
    }
    
};

In [ ]:
%%writefile simpleclass.pxd
# distutils: language = c++

cdef extern from "SimpleClass.hpp":
    cdef cppclass Simple:
        Simple(int x)
        int get()

In [ ]:
%%writefile cythonclass.pyx
# distutils: language = c++

from simpleclass cimport Simple as cSimple

cdef class Simple:
    cdef cSimple *cself
    
    def __cinit__(self, int x):
        self.cself = new cSimple(x)
    
    def get(self):
        return self.cself.get()
    
    def __dealloc__(self):
        del self.cself

In [ ]:
%%writefile setup.py

from setuptools import setup, Extension

module1 = Extension('pysimple',
                    sources=['cythonclass.pyx'],
                    language='c++'
                   )

setup(name='pysimple', ext_modules=[module1])

You can use `cythonize` on the command line to build the wrapper and look at it, but setuptools now natively supports Cython! If you do this manually, a pyproject.toml (PEP 517/518 build) to ensure Cython is available in the build environment (and as always, [distibute wheels](https://iscinumpy.gitlab.io/post/azure-devops-python-wheels)). I'll be writing nice guides on Scikit-HEP at some point in the near future that may be useful even if you are not in HEP.

In [ ]:
!python setup.py build_ext --inplace

In [ ]:
import pysimple
x = pysimple.Simple(3)
x.get()